In [1]:
!pip install datasets
!pip install transformers
!pip install sentencepiece
!pip install transformers[torch]`
!pip install sacrebleu
!pip install evaluate
!pip install sacrebleu
!pip install accelerate -U
!pip install gradio
!pip install typing-extensions==3.10.0.2

!pip install kaleido cohere  openai tiktoken typing-extensions==4.5.0

/bin/bash: -c: line 1: unexpected EOF while looking for matching ``'
/bin/bash: -c: line 2: syntax error: unexpected end of file
  Attempting uninstall: typing-extensions
    Found existing installation: typing_extensions 4.5.0
    Uninstalling typing_extensions-4.5.0:
      Successfully uninstalled typing_extensions-4.5.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gradio 4.21.0 requires pydantic>=2.0, but you have pydantic 1.10.14 which is incompatible.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 394.9/394.9 kB 7.1 MB/s eta 0:00:00
  Attempting uninstall: pydantic
    Found existing installation: pydantic 1.10.14
    Uninstalling pydantic-1.10.14:
      Successfully uninstalled pydantic-1.10.14
  Using cached typing_extensions-3.10.0.2-py3-none-any.whl (26 kB)
  Attempting uninstall: typing-extensions
    Found existing installation: typing_extensions 4

In [2]:
from datasets import load_dataset
dataset = load_dataset("cfilt/iitb-english-hindi")


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


In [3]:
type(dataset)

datasets.dataset_dict.DatasetDict

In [4]:
dataset['train'][1]['translation']


{'en': 'Accerciser Accessibility Explorer',
 'hi': 'एक्सेर्साइसर पहुंचनीयता अन्वेषक'}

In [5]:
dataset['validation'][0]['translation']

{'en': 'Students of the Dattatreya city Municipal corporation secondary school demonstrated their imagination power by creating the fictitious fort "Duttgarh".',
 'hi': "महानगर पालिका अंतर्गत दत्तात्रय नगर माध्यमिक स्कूल के विद्यार्थियों ने काल्पनिक किला 'दत्तगढ़' बनाकर अपनी कल्पनाशक्ति का परिचय दिया।"}

In [6]:


# Load model directly
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

tokenizer = AutoTokenizer.from_pretrained("Helsinki-NLP/opus-mt-en-hi")
model = AutoModelForSeq2SeqLM.from_pretrained("Helsinki-NLP/opus-mt-en-hi")


/usr/local/lib/python3.10/dist-packages/transformers/models/marian/tokenization_marian.py:197: UserWarning: Recommended: pip install sacremoses.
  warnings.warn("Recommended: pip install sacremoses.")


In [7]:
max_length = 256

def preprocess_function(examples):
  inputs = [ex["en"] for ex in examples["translation"]]
  targets = [ex["hi"] for ex in examples["translation"]]

  model_inputs = tokenizer(inputs, max_length=max_length, truncation=True)
  labels = tokenizer(targets,max_length=max_length, truncation=True)
  model_inputs["labels"] = labels["input_ids"]

  return model_inputs


In [8]:
tokenized_datasets_validation = dataset['validation'].map(
	preprocess_function,
	batched= True,
	remove_columns=dataset["validation"].column_names,
	batch_size = 2
)

tokenized_datasets_test = dataset['test'].map(
	preprocess_function,
	batched= True,
	remove_columns=dataset["test"].column_names,
	batch_size = 2)


In [9]:
tokenized_datasets_test[0]

{'input_ids': [238, 3078, 3613, 21, 85, 3869, 22, 0],
 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1],
 'labels': [44,
  3605,
  2703,
  716,
  130,
  44,
  716,
  260,
  428,
  44,
  1056,
  174,
  549,
  44,
  1400,
  1185,
  800,
  26150,
  44,
  1400,
  4182,
  716,
  1185,
  680,
  22,
  0]}

In [10]:
from transformers import DataCollatorForSeq2Seq

data_collator = DataCollatorForSeq2Seq(tokenizer, model=model)


In [11]:
print(model)


MarianMTModel(
  (model): MarianModel(
    (shared): Embedding(61950, 512, padding_idx=61949)
    (encoder): MarianEncoder(
      (embed_tokens): Embedding(61950, 512, padding_idx=61949)
      (embed_positions): MarianSinusoidalPositionalEmbedding(512, 512)
      (layers): ModuleList(
        (0-5): 6 x MarianEncoderLayer(
          (self_attn): MarianAttention(
            (k_proj): Linear(in_features=512, out_features=512, bias=True)
            (v_proj): Linear(in_features=512, out_features=512, bias=True)
            (q_proj): Linear(in_features=512, out_features=512, bias=True)
            (out_proj): Linear(in_features=512, out_features=512, bias=True)
          )
          (self_attn_layer_norm): LayerNorm((512,), eps=1e-05, elementwise_affine=True)
          (activation_fn): SiLU()
          (fc1): Linear(in_features=512, out_features=2048, bias=True)
          (fc2): Linear(in_features=2048, out_features=512, bias=True)
          (final_layer_norm): LayerNorm((512,), eps=1e-05

In [12]:
# Access the layers and freeze the specified number of layers
# Specify the number of layers to freeze from the end

for parameter in model.parameters():
	parameter.requires_grad = True
num_layers_to_freeze = 10 # Adjust as needed
for layer_index, layer in enumerate(model.model.encoder.layers):
	print
	if layer_index < len(model.model.encoder.layers) - num_layers_to_freeze:
		for parameter in layer.parameters():
			parameter.requires_grad = False

num_layers_to_freeze = 10 # Adjust as needed
for layer_index, layer in enumerate(model.model.decoder.layers):
	print
	if layer_index < len(model.model.encoder.layers) - num_layers_to_freeze:
		for parameter in layer.parameters():
			parameter.requires_grad = False


In [13]:
import evaluate

metric = evaluate.load("sacrebleu")

import numpy as np


def compute_metrics(eval_preds):
	preds, labels = eval_preds
	# In case the model returns more than the prediction logits
	if isinstance(preds, tuple):
		preds = preds[0]

	decoded_preds = tokenizer.batch_decode(preds, skip_special_tokens=True)

	# Replace -100s in the labels as we can't decode them
	labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
	decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

	# Some simple post-processing
	decoded_preds = [pred.strip() for pred in decoded_preds]
	decoded_labels = [[label.strip()] for label in decoded_labels]

	result = metric.compute(predictions=decoded_preds, references=decoded_labels)
	return {"bleu": result["score"]}


In [21]:
import torch
# Specify the device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Move the model to the specified device
model.to(device)


#Comment if not using cuda

MarianMTModel(
  (model): MarianModel(
    (shared): Embedding(61950, 512, padding_idx=61949)
    (encoder): MarianEncoder(
      (embed_tokens): Embedding(61950, 512, padding_idx=61949)
      (embed_positions): MarianSinusoidalPositionalEmbedding(512, 512)
      (layers): ModuleList(
        (0-5): 6 x MarianEncoderLayer(
          (self_attn): MarianAttention(
            (k_proj): Linear(in_features=512, out_features=512, bias=True)
            (v_proj): Linear(in_features=512, out_features=512, bias=True)
            (q_proj): Linear(in_features=512, out_features=512, bias=True)
            (out_proj): Linear(in_features=512, out_features=512, bias=True)
          )
          (self_attn_layer_norm): LayerNorm((512,), eps=1e-05, elementwise_affine=True)
          (activation_fn): SiLU()
          (fc1): Linear(in_features=512, out_features=2048, bias=True)
          (fc2): Linear(in_features=2048, out_features=512, bias=True)
          (final_layer_norm): LayerNorm((512,), eps=1e-05

In [22]:
from transformers import Seq2SeqTrainingArguments

model.to(device)
training_args = Seq2SeqTrainingArguments(
	f"finetuned-nlp-en-hi",
	gradient_checkpointing=True,
	per_device_train_batch_size=32,
	learning_rate=1e-5,
	warmup_steps=2,
	max_steps=100, #Hyperparamert . Try putting this as 1000 / 2000
	fp16=False,
	optim='adafactor',
	per_device_eval_batch_size=16,
	metric_for_best_model="eval_bleu",
	predict_with_generate=True,
	push_to_hub=False
)


In [16]:
from transformers import Seq2SeqTrainer

trainer = Seq2SeqTrainer(
	model,
	training_args,
	train_dataset=tokenized_datasets_test,
	eval_dataset=tokenized_datasets_validation,
	data_collator=data_collator,
	tokenizer=tokenizer,
	compute_metrics=compute_metrics,
)

trainer.train()


/usr/local/lib/python3.10/dist-packages/accelerate/accelerator.py:432: FutureWarning: Passing the following arguments to `Accelerator` is deprecated and will be removed in version 1.0 of Accelerate: dict_keys(['dispatch_batches', 'split_batches', 'even_batches', 'use_seedable_sampler']). Please pass an `accelerate.DataLoaderConfiguration` instead: 
dataloader_config = DataLoaderConfiguration(dispatch_batches=None, split_batches=False, even_batches=True, use_seedable_sampler=True)
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:460: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(


Step,Training Loss


TrainOutput(global_step=100, training_loss=4.149725036621094, metrics={'train_runtime': 117.103, 'train_samples_per_second': 27.326, 'train_steps_per_second': 0.854, 'total_flos': 55543000596480.0, 'train_loss': 4.149725036621094, 'epoch': 1.27})

In [17]:



output_dir = "./trained_model"
model.save_pretrained(output_dir)


Some non-default generation parameters are set in the model config. These should go into a GenerationConfig file (https://huggingface.co/docs/transformers/generation_strategies#save-a-custom-decoding-strategy-with-your-model) instead. This warning will be raised to an exception in v4.41.
Non-default generation parameters: {'max_length': 512, 'num_beams': 4, 'bad_words_ids': [[61949]], 'forced_eos_token_id': 0}


In [18]:
from google.colab import files

# Specify the directory where the fine-tuned model is saved
output_dir = "/content/trained_model"

# Zip the model directory
zip_file_name = "trained.zip"
!zip -r $zip_file_name $output_dir

# Download the zip file
files.download(zip_file_name)


  adding: content/trained_model/ (stored 0%)
  adding: content/trained_model/config.json (deflated 61%)
  adding: content/trained_model/generation_config.json (deflated 43%)
  adding: content/trained_model/model.safetensors (deflated 7%)


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [23]:
# import gradio as gr


def translate(text):
  inputs = tokenizer(text, return_tensors="pt").to(device)
  translated_tokens = model.generate(**inputs, max_length=256)
  results = tokenizer.batch_decode(translated_tokens, skip_special_tokens=True)[0]
  return results



print(translate("Nice to Meet You"))
# #Creating the User Interface Space
# interface = gr.Interface(fn=translate,inputs=gr.Textbox(lines=2, placeholder='Text to translate'),
# 						outputs='text')
# #launching the interface
# interface.launch()


/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:460: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:90: UserWarning: None of the inputs have requires_grad=True. Gradients will be None
  warnings.warn(
`use_cache=True` is incompatible with gradient checkpointing. Setting `use_cache=False`...


आप से मिलकर अच्छा लगा


###Model 2

In [ ]:
!nvidia-smi

Mon Mar 18 15:37:31 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla T4                       Off | 00000000:00:04.0 Off |                    0 |
| N/A   33C    P8               9W /  70W |      0MiB / 15360MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [ ]:
!pip install datasets transformers[sentencepiece] sacrebleu -q

In [ ]:
import os
import sys
import transformers
import tensorflow as tf
from datasets import load_dataset
from transformers import AutoTokenizer
from transformers import TFAutoModelForSeq2SeqLM, DataCollatorForSeq2Seq
from transformers import AdamWeightDecay
from transformers import AutoTokenizer, TFAutoModelForSeq2SeqLM

In [ ]:
model_checkpoint = "Helsinki-NLP/opus-mt-en-hi"

In [ ]:
raw_datasets = load_dataset("cfilt/iitb-english-hindi")

In [ ]:
raw_datasets

DatasetDict({
    train: Dataset({
        features: ['translation'],
        num_rows: 1659083
    })
    validation: Dataset({
        features: ['translation'],
        num_rows: 520
    })
    test: Dataset({
        features: ['translation'],
        num_rows: 2507
    })
})

In [ ]:
raw_datasets['train'][2]

{'translation': {'en': 'The default plugin layout for the bottom panel',
  'hi': 'निचले पटल के लिए डिफोल्ट प्लग-इन खाका'}}

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

tokenizer_config.json:   0%|          | 0.00/44.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.39k [00:00<?, ?B/s]

source.spm:   0%|          | 0.00/812k [00:00<?, ?B/s]

target.spm:   0%|          | 0.00/1.07M [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/2.10M [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/models/marian/tokenization_marian.py:197: UserWarning: Recommended: pip install sacremoses.
  warnings.warn("Recommended: pip install sacremoses.")


In [ ]:
tokenizer("Hello, this is a sentence!")

{'input_ids': [12110, 2, 90, 23, 19, 8800, 61, 0], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1]}

In [ ]:
tokenizer.save_pretrained("model_tokenizor")

('model_tokenizor/tokenizer_config.json',
 'model_tokenizor/special_tokens_map.json',
 'model_tokenizor/vocab.json',
 'model_tokenizor/source.spm',
 'model_tokenizor/target.spm',
 'model_tokenizor/added_tokens.json')

In [ ]:
# from google.colab import files
# import shutil

# # Zip the directory
# shutil.make_archive("/content/model_tokenizor", "zip", "model_tokenizor")

# # Download the zip file
# files.download("model_tokenizor.zip")


In [ ]:
with tokenizer.as_target_tokenizer():
    print(tokenizer(["एक्सेर्साइसर पहुंचनीयता अन्वेषक"]))

{'input_ids': [[26618, 16155, 346, 33383, 0]], 'attention_mask': [[1, 1, 1, 1, 1]]}


/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:3892: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(


In [ ]:
max_input_length = 128
max_target_length = 128

source_lang = "en"
target_lang = "hi"


def preprocess_function(examples):
    inputs = [ex[source_lang] for ex in examples["translation"]]
    targets = [ex[target_lang] for ex in examples["translation"]]
    model_inputs = tokenizer(inputs, max_length=max_input_length, truncation=True)

    # Setup the tokenizer for targets
    with tokenizer.as_target_tokenizer():
        labels = tokenizer(targets, max_length=max_target_length, truncation=True)

    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

In [ ]:
preprocess_function(raw_datasets["train"][:2])

{'input_ids': [[3872, 85, 2501, 132, 15441, 36398, 0], [32643, 28541, 36253, 0]], 'attention_mask': [[1, 1, 1, 1, 1, 1, 1], [1, 1, 1, 1]], 'labels': [[63, 2025, 18, 16155, 346, 20311, 24, 2279, 679, 0], [26618, 16155, 346, 33383, 0]]}

In [ ]:
tokenized_datasets = raw_datasets.map(preprocess_function, batched=True)

Map:   0%|          | 0/1659083 [00:00<?, ? examples/s]

Map:   0%|          | 0/520 [00:00<?, ? examples/s]

Map:   0%|          | 0/2507 [00:00<?, ? examples/s]

In [ ]:
model = TFAutoModelForSeq2SeqLM.from_pretrained(model_checkpoint)

tf_model.h5:   0%|          | 0.00/306M [00:00<?, ?B/s]

All model checkpoint layers were used when initializing TFMarianMTModel.

All the layers of TFMarianMTModel were initialized from the model checkpoint at Helsinki-NLP/opus-mt-en-hi.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFMarianMTModel for predictions without further training.


generation_config.json:   0%|          | 0.00/293 [00:00<?, ?B/s]

In [ ]:
data_collator = DataCollatorForSeq2Seq(tokenizer, model=model, return_tensors="tf")

In [ ]:
generation_data_collator = DataCollatorForSeq2Seq(tokenizer, model=model, return_tensors="tf", pad_to_multiple_of=128)

In [ ]:
tokenized_datasets

DatasetDict({
    train: Dataset({
        features: ['translation', 'input_ids', 'attention_mask', 'labels'],
        num_rows: 1659083
    })
    validation: Dataset({
        features: ['translation', 'input_ids', 'attention_mask', 'labels'],
        num_rows: 520
    })
    test: Dataset({
        features: ['translation', 'input_ids', 'attention_mask', 'labels'],
        num_rows: 2507
    })
})

In [ ]:
tokenized_datasets['train'][0:3]

{'translation': [{'en': 'Give your application an accessibility workout',
   'hi': 'अपने अनुप्रयोग को पहुंचनीयता व्यायाम का लाभ दें'},
  {'en': 'Accerciser Accessibility Explorer',
   'hi': 'एक्सेर्साइसर पहुंचनीयता अन्वेषक'},
  {'en': 'The default plugin layout for the bottom panel',
   'hi': 'निचले पटल के लिए डिफोल्ट प्लग-इन खाका'}],
 'input_ids': [[3872, 85, 2501, 132, 15441, 36398, 0],
  [32643, 28541, 36253, 0],
  [81, 1724, 1907, 6461, 34, 4, 5571, 10061, 0]],
 'attention_mask': [[1, 1, 1, 1, 1, 1, 1],
  [1, 1, 1, 1],
  [1, 1, 1, 1, 1, 1, 1, 1, 1]],
 'labels': [[63, 2025, 18, 16155, 346, 20311, 24, 2279, 679, 0],
  [26618, 16155, 346, 33383, 0],
  [18464, 12422, 6, 39, 22433, 10076, 69, 4553, 7788, 0]]}

In [ ]:
batch_size = 16
learning_rate = 2e-5
weight_decay = 0.01
num_train_epochs = 1

In [ ]:
train_dataset = model.prepare_tf_dataset(
    tokenized_datasets["test"],
    batch_size=batch_size,
    shuffle=True,
    collate_fn=data_collator,
)
validation_dataset = model.prepare_tf_dataset(
    tokenized_datasets["validation"],
    batch_size=batch_size,
    shuffle=False,
    collate_fn=data_collator,
)

generation_dataset = model.prepare_tf_dataset(
    tokenized_datasets["validation"],
    batch_size=8,
    shuffle=False,
    collate_fn=generation_data_collator,
)

In [ ]:
optimizer = AdamWeightDecay(learning_rate=learning_rate, weight_decay_rate=weight_decay)
model.compile(optimizer=optimizer)

In [ ]:
model.fit(train_dataset, validation_data=validation_dataset, epochs=10)

Epoch 1/10
156/156 [==============================] - 48s 307ms/step - loss: 3.3253 - val_loss: 3.8751
Epoch 2/10
156/156 [==============================] - 48s 308ms/step - loss: 3.0133 - val_loss: 3.8286
Epoch 3/10
156/156 [==============================] - 49s 312ms/step - loss: 2.7690 - val_loss: 3.8189
Epoch 4/10
156/156 [==============================] - 46s 298ms/step - loss: 2.5647 - val_loss: 3.8188
Epoch 5/10
156/156 [==============================] - 48s 310ms/step - loss: 2.3728 - val_loss: 3.8263
Epoch 6/10
156/156 [==============================] - 48s 308ms/step - loss: 2.2004 - val_loss: 3.8366
Epoch 7/10
156/156 [==============================] - 46s 293ms/step - loss: 2.0517 - val_loss: 3.8369
Epoch 8/10
156/156 [==============================] - 50s 318ms/step - loss: 1.9080 - val_loss: 3.8657
Epoch 9/10
156/156 [==============================] - 49s 317ms/step - loss: 1.7752 - val_loss: 3.8890
Epoch 10/10
156/156 [==============================] - 53s 337ms/step - l

In [ ]:
model.save_pretrained("tf_model/")

Some non-default generation parameters are set in the model config. These should go into a GenerationConfig file (https://huggingface.co/docs/transformers/generation_strategies#save-a-custom-decoding-strategy-with-your-model) instead. This warning will be raised to an exception in v4.41.
Non-default generation parameters: {'max_length': 512, 'num_beams': 4, 'bad_words_ids': [[61949]], 'forced_eos_token_id': 0}


In [ ]:
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)
model = TFAutoModelForSeq2SeqLM.from_pretrained("tf_model/")

/usr/local/lib/python3.10/dist-packages/transformers/models/marian/tokenization_marian.py:197: UserWarning: Recommended: pip install sacremoses.
  warnings.warn("Recommended: pip install sacremoses.")
All model checkpoint layers were used when initializing TFMarianMTModel.

All the layers of TFMarianMTModel were initialized from the model checkpoint at tf_model/.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFMarianMTModel for predictions without further training.


In [ ]:


input_text  = "My name is Shrey. Nice to meet You"

tokenized = tokenizer([input_text], return_tensors='np')
out = model.generate(**tokenized, max_length=128)
print(out)

tf.Tensor(
[[61949   500   179  6652   130     5    40  4265  2510   470   565     0
  61949]], shape=(1, 13), dtype=int32)


In [ ]:
with tokenizer.as_target_tokenizer():
    print(tokenizer.decode(out[0], skip_special_tokens=True))

मेरा नाम श्रीी है। आपसे मिलकर अच्छा लगा


/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:3892: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(
